In [6]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.0/297.0 kB 3.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 2.5 MB/s eta 0:00:00a 0:00:01
  Created wheel for ipdb: filename=ipdb-0.13.9-py3-none-any.whl size=11628 sha256=9af653c28629ec266d5ca45a942b4f411c045641bbad93790c81f4b4bded9887
  Stored in directory: /home/jovyan/.cache/pip/wheels/54/02/23/574616a4405110c3a9ccf58c81a7bdea1145360f54adb240a5
Successfully built ipdb


In [8]:
import time
import pytest
from freezegun import freeze_time

from main import create_exchange, create_staking_rewards
from lp import Factory

# Piscinas de liquidez (liquidity pool)

- Based on Uniswap v1 contracts
- Uniswap is made up of a series of ETH-ERC20 exchange contracts. There is exactly one exchange contract per ERC20 token. If a token does not yet have an exchange it can be created by anyone using the Uniswap factory contract. The factory serves as a public registry and is used to look up all token and exchange addresses added to the system.
- Each exchange holds reserves of both ETH and its associated ERC20 token. Anyone can become a liquidity provider on an exchange and contribute to its reserves. This is different than buying or selling; it requires depositing an equivalent value of both ETH and the relevant ERC20 token.
- Liquidity is pooled across all providers and an internal "pool token" (ERC20) is used to track each providers relative contribution. Pool tokens are minted when liquidity is deposited into the system and can be burned at any time to withdraw a proportional share of the reserves.
- Exchange contracts are automated market makers between an ETH-ERC20 pair. Traders can swap between the two in either direction by adding to the liquidity reserve of one and withdrawing from the reserve of the other.
- Uniswap uses a "constant product" market making formula which sets the exchange rate based off of the relative size of the ETH and ERC20 reserves, and the amount with which an incoming trade shifts this ratio.
- A small liquidity provider fee (0.30%) is taken out of each trade and added to the reserves.
- This functions as a payout to liquidity providers that is collected when they burn their pool tokens to withdraw their portion of total reserves.


## Referências
- https://docs.uniswap.org/protocol/V1/reference/interfaces
- https://docs.uniswap.org/protocol/V1/introduction
- https://hackmd.io/@HaydenAdams/HJ9jLsfTz?type=view#%F0%9F%A6%84-Uniswap-Whitepaper
- https://github.com/Uniswap/v1-contracts/blob/c10c08d81d6114f694baa8bd32f555a40f6264da/contracts/uniswap_exchange.vy
- https://github.dev/Uniswap/v2-core/blob/master/contracts/UniswapV2Factory.sol
- https://github.dev/Uniswap/v2-periphery/blob/master/contracts/UniswapV2Router02.sol
- https://docs.uniswap.org/protocol/V1/guides/pool-liquidity
- https://docs.uniswap.org/protocol/V2/concepts/core-concepts/pools
- https://uniswap.org/whitepaper.pdf
- https://betterprogramming.pub/uniswap-smart-contract-breakdown-ea20edf1a0ff
- https://coinsbench.com/erc20-smart-contract-breakdown-9dab65cec671
- https://betterprogramming.pub/uniswap-smart-contract-breakdown-part-2-b9ea2fca65d1
- https://medium.com/scalar-capital/uniswap-a-unique-exchange-f4ef44f807bf
- https://ethereum.org/pt-br/developers/tutorials/uniswap-v2-annotated-code/
- https://ethereum.org/en/developers/tutorials/uniswap-v2-annotated-code/#add-liquidity-flow
- https://jeiwan.net/posts/programming-defi-uniswapv2-1/
- https://docs.uniswap.org/protocol/V2/concepts/protocol-overview/ecosystem-participants

In [12]:
fabrica = Factory("Fábrica de piscinas de liquidez", "0x1")
piscinal_de_liquidez = create_exchange(name="test-coin", address="0x111", symbol="TST1", _factory=fabrica)

In [13]:
piscinal_de_liquidez.info()
piscinal_de_liquidez.doc()

Exchange test-coin/ETH (TST1)
Moedas: test-coin/ETH
Reservas: test-coin = 0 | ETH = 0
Invariante: 0 * 0 = 0

Funcionalidades disponíveis:
- Adicionar liquidez
- Remover liquidez
- Trocar tokens



In [14]:
eth_amount = 50000
tst_amount = 50000
piscinal_de_liquidez.add_liquidity(
    _from="rsarai",
    balance0=eth_amount,
    balance1=tst_amount,
    balance0Min=eth_amount,
    balance1Min=tst_amount
)
piscinal_de_liquidez.info()

Exchange test-coin/ETH (TST1)
Moedas: test-coin/ETH
Reservas: test-coin = 50000 | ETH = 50000
Invariante: 50000 * 50000 = 2500000000



In [15]:
piscinal_de_liquidez.simulate_transaction(amount_t0=50)
piscinal_de_liquidez.simulate_transaction(amount_t0=90)
piscinal_de_liquidez.simulate_transaction(amount_t0=300)
piscinal_de_liquidez.simulate_transaction(amount_t0=1000)
piscinal_de_liquidez.simulate_transaction(amount_t0=50000)

50 test-coin recebe 49.95 ETH
90 test-coin recebe 89.84 ETH
300 test-coin recebe 298.21 ETH
1000 test-coin recebe 980.39 ETH
50000 test-coin recebe 25000.0 ETH


In [17]:
with pytest.raises(Exception, match="UniswapV2Router: INSUFFICIENT_OUTPUT_AMOUNT"):
    piscinal_de_liquidez.swapExactTokensForTokens(amount0_in=50, amount1_out_min=375, to="rsarai")

In [18]:
with pytest.raises(Exception, match="UniswapV2Router: INSUFFICIENT_OUTPUT_AMOUNT"):
    piscinal_de_liquidez.swapExactTokensForTokens(amount0_in=50, amount1_out_min=50, to="rsarai")

In [20]:
print(piscinal_de_liquidez.swapExactTokensForTokens(amount0_in=50, amount1_out_min=49, to="rsarai"), '\n')
piscinal_de_liquidez.info()

49.95004995004995 

Exchange test-coin/ETH (TST1)
Moedas: test-coin/ETH
Reservas: test-coin = 50050 | ETH = 49950.04995004995
Invariante: 50050 * 49950.04995004995 = 2500000000.0



In [21]:
piscinal_de_liquidez.simulate_transaction(amount_t0=400)    # 396.04 ETH
print(piscinal_de_liquidez.swapExactTokensForTokens(amount0_in=400, amount1_out_min=396, to="rsarai"))
piscinal_de_liquidez.info()

400 test-coin recebe 396.04 ETH
396.03607492606506
Exchange test-coin/ETH (TST1)
Moedas: test-coin/ETH
Reservas: test-coin = 50450 | ETH = 49554.013875123885
Invariante: 50450 * 49554.013875123885 = 2500000000.0



In [22]:
piscinal_de_liquidez.simulate_transaction(amount_t0=400)    # 389.81 ETH
print(piscinal_de_liquidez.swapExactTokensForTokens(amount0_in=400, amount1_out_min=389, to="rsarai"))
piscinal_de_liquidez.info()

400 test-coin recebe 389.81 ETH
389.8054188800305
Exchange test-coin/ETH (TST1)
Moedas: test-coin/ETH
Reservas: test-coin = 50850 | ETH = 49164.208456243854
Invariante: 50850 * 49164.208456243854 = 2500000000.0



## Important code to check
- lp.py#\_\_init\_\_
- lp.py#get_amount_out
- lp.py#ERC20

# Mineração de liquidez (liquidity mining)
- This functionality is enabled by the StakingRewards contract

In [59]:
# Setup same as before
factory = Factory("ETH pool factory", "0x2")
lp = create_exchange("LM", "0x112", "LM2", factory)

eth_amount = 50000
tst_amount = 50000
lp.add_liquidity("rsarai", eth_amount, tst_amount, eth_amount, tst_amount)

(50000, 50000)

In [60]:
# creates staking rewards
with freeze_time('2022-08-20 14:49:07'):
    st = create_staking_rewards(lp, 1_000_000)
    st.doc()

Funcionalidades disponíveis: 
- [public] Stake
- [public] Withdraw
- [public] Get rewards
- [private] Add rewards


In [61]:
st.info()

Balanças:  {'deployer': 0}
Reward rate:  0.031709791983764585
Total staked:  0
Última atualização:  1661006947.0


In [62]:
with pytest.raises(Exception, match="Not enough provided liquidity"):
    st.stake("random", 1000)

In [63]:
with freeze_time('2022-08-20 14:49:07'):
    st.stake("rsarai", 49990.0)
    st.info()

Balanças:  {'deployer': 0, 'rsarai': 49990.0}
Reward rate:  0.031709791983764585
Total staked:  49990.0
Última atualização:  1661006947.0


In [64]:
st.info()

Balanças:  {'deployer': 0, 'rsarai': 49990.0}
Reward rate:  0.031709791983764585
Total staked:  49990.0
Última atualização:  1661006947.0


In [65]:
with freeze_time('2022-08-20 14:49:12'):
    print("Quantidade de recompensas em 5s", round(st.earned("rsarai"), 3))

Quantidade de recompensas em 5s 0.159


In [66]:
with freeze_time('2022-08-20 14:49:17'):
    print("Quantidade de recompensas em 10s", round(st.earned("rsarai"), 3))

Quantidade de recompensas em 10s 0.317


In [67]:
with freeze_time('2023-08-20 14:49:12'):
    print("Quantidade de recompensas em 1 ano", st.earned("rsarai"))

Quantidade de recompensas em 1 ano 1000000.0


In [68]:
# Setup same as before
factory = Factory("ETH pool factory", "0x2")
lp = create_exchange("LM", "0x112", "LM2", factory)

eth_amount = 50000
tst_amount = 50000
lp.add_liquidity("rsarai", eth_amount, tst_amount, eth_amount, tst_amount)

eth_amount = 50000
tst_amount = 50000
lp.add_liquidity("garrincha", eth_amount, tst_amount, eth_amount, tst_amount)

(50000, 50000.0)

In [69]:
with freeze_time('2022-08-20 14:49:07'):
    st = create_staking_rewards(lp, 1_000_000)
    st.stake("rsarai", 49990.0)

with freeze_time('2022-08-20 14:49:12'):
    st.stake("garrincha", 25000.0)

In [70]:
st.info()

Balanças:  {'deployer': 0, 'rsarai': 49990.0, 'garrincha': 25000.0}
Reward rate:  0.031709791983764585
Total staked:  74990.0
Última atualização:  1661006952.0


In [71]:
with freeze_time('2022-08-20 14:49:17'):
    print("Quantidade de recompensas em 10s", st.earned("rsarai"))
    print("Quantidade de recompensas em 10s", st.earned("garrincha"))

Quantidade de recompensas em 10s 0.2113845181048662
Quantidade de recompensas em 10s 0.10571340173277965


## Important code to check
- staking_rewards.py#\_\_init\_\_
- staking_rewards.py#earned